## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [3]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [4]:
import yaml

In [5]:
from IPython.html.widgets import FloatProgress
from IPython.display import display

/Users/gilles/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [6]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [7]:
saut="\n"

### Préparation des matrices de traits

In [8]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [9]:
sampleNumber="01-N"
genDigraphe=False
genGraphe=False
samplePrefix="MGC-160104-%s-ext3-derivations-SILVER"%sampleNumber
sampleFile=samplePrefix+".csv"
analysisPrefix="MGC-160104-%s-ext3-derivations-SILVER"%sampleNumber
logfile_name=analysisPrefix+samplePrefix+".log"
logfile = codecs.open("2015-Data/"+logfile_name,mode='w',encoding="utf8")

In [10]:
paradigmes=pd.read_csv("2015-Data/"+sampleFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]
paradigmes=paradigmes.dropna(axis=1,how='all')

In [11]:
phonologicalMap=analysisPrefix[-2:]
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [12]:
cliqueFile="2015-Data/"+analysisPrefix+'-Network.pkl'
#paradigmes


In [13]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [14]:
goldCases=paradigmes.columns.tolist()
goldCases.remove("lexeme")
#goldCases

- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [15]:
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")
#sampleCases

### Définition des objets pour la gestion des règles

In [16]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print ("le numéro de forme n'est pas dans [1,2]",n,file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print (forme, file=logfile) 
            print ("pas de patron", file=logfile)
        return sortieForme
        

### Comparer les cases analysées avec l'ensemble de toutes les cases

In [17]:
with open("2015-Data/"+analysisPrefix+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)

In [18]:
analyseCases=list(set([case for (case,autre) in resultatsLecture.keys() if autre in ["FP"]]))
print (sorted(analyseCases))
#    analyseCases=sampleCases

[u'FP', u'FS', u'ai1P', u'ai1S', u'ai2P', u'ai2S', u'ai3P', u'ai3S', u'fi1P', u'fi1S', u'fi2P', u'fi2S', u'fi3P', u'fi3S', u'ii1P', u'ii1S', u'ii2P', u'ii2S', u'ii3P', u'ii3S', u'inf', u'is1P', u'is1S', u'is3P', u'is3S', u'pI1P', u'pI2P', u'pI2S', u'pP', u'pc1P', u'pc1S', u'pc2P', u'pc2S', u'pc3P', u'pc3S', u'pi1P', u'pi1S', u'pi2P', u'pi2S', u'pi3P', u'pi3S', u'ppFP', u'ppFS', u'ppMP', u'ppMS', u'ps1P', u'ps1S', u'ps2P', u'ps2S', u'ps3P', u'ps3S']


#### Calculer le score de la clique

In [19]:
def cliqueScore(clique,graph):
    score=0
    for (depart,arrivee) in it.combinations_with_replacement(clique,2):
        score+=graph[depart][arrivee]["weight"]
    return score

## Génération des formes

In [20]:
basesDerives=pd.read_csv("2015-Data/MGC-160104-01-N-basesDerives.csv",sep=";",encoding="utf8")

In [21]:
with open("2015-Data/"+analysisPrefix+'-Network.pkl', 'rb') as input:
    infoCliques=pickle.load(input)
cliques=infoCliques["cliques"]
cliquesScores=infoCliques["cliquesScores"]
cliquesListes=infoCliques["cliquesListes"]

In [22]:
cliques

[[u'absoudre-absOlvE-ii1S',
  u'absoudre-absOlv-pI2S',
  u'absoudre-absOlvE-ii3S',
  u'absoudre-absOlv\xf4-pi1P',
  u'absoudre-absOlve-ppMP',
  u'absoudre-absOlv-pi3P',
  u'absoudre-absOlvEr-ai3P',
  u'absoudre-absOlva-ai3S',
  u'absoudre-absOlve-ppFS',
  u'absoudre-absOlve-pi2P',
  u'absoudre-absOlvE-ai1S',
  u'absoudre-absOlv-ps1S',
  u'absoudre-absOlv-pi2S',
  u'absoudre-absOlv-pi1S'],
 [u'absoudre-absOlvE-ii1S',
  u'absoudre-absOlv-pI2S',
  u'absoudre-absOlvE-ii3S',
  u'absoudre-absOlv\xf4-pi1P',
  u'absoudre-absOlve-ppMP',
  u'absoudre-absOlv-pi3P',
  u'absoudre-absOlvEr-ai3P',
  u'absoudre-absOlva-ai3S',
  u'absoudre-absOlve-ppFS',
  u'absoudre-absOlve-pi2P',
  u'absoudre-absOlvE-ai1S',
  u'absoudre-absOlvE-ii3P'],
 [u'absoudre-absOlvE-ii1S',
  u'absoudre-absOlv-pI2S',
  u'absoudre-absOlvE-ii3S',
  u'absoudre-absOlv\xf4-pi1P',
  u'absoudre-absOlve-ppMP',
  u'absoudre-absOlv-pi3P',
  u'absoudre-absOlvEr-ai3P',
  u'absoudre-absOlva-ai3S',
  u'absoudre-absOlve-ppFS',
  u'absoudre-ab